# Lineære Systemer av Differensialligninger (MIP 11.1)

## **Matriser og system av differensialligninger**

Vi har sett på systemer av førsteordens differensialligninger på formen
$$
\begin{align}
  x'(t) &= f(t,x(t), y(t)), \newline
  y'(t) &= g(t,x(t), y(t))
\end{align}
$$
for funksjoner
$$
\begin{align}
\mathbb{R}^3 &\xrightarrow f \mathbb{R}, \newline
(t, x, y) &\mapsto f(t, x, y)
\end{align}
$$
og
$$
\begin{align}
\mathbb{R}^3 &\xrightarrow g \mathbb{R}, \newline
(t, x, y) &\mapsto g(t, x, y).
\end{align}
$$

Vi kan tenke på dette som en ligning mellom vektorer i $\mathbb{R}^2$.

Lar vi $\vec v(t) = \begin{bmatrix}x(t)\\ y(t)\end{bmatrix}$ og $\vec F(t, \vec v(t)) = \begin{bmatrix}f(t, x(t), y(t))\\ g(t, x(t), y(t))\end{bmatrix}$, kan vi skrive systemet som
$$
\begin{align}
  \vec v'(t) &= \vec F(t, \vec v(t))
\end{align}
$$
der
$\vec v'(t) = \begin{bmatrix}x'(t)\\y'(t)\end{bmatrix}$
er den koordinatvis deriverte til $\vec v(t)$.

Dette er en kompakt måte å skrive systemet på, og vi kan bruke vektorregning for å løse det.

---

Vi vil nå se på en enkel versjon av slike systemer der det finnes tall $p, q, r$ og $s$ slik at systemet er på formen
$$
\begin{align}
  x'(t) &= p \cdot x(t) +  q \cdot y(t) \\
  y'(t) &= r \cdot x(t) + s \cdot y(t))
\end{align}
$$


Hvis vi lar
$A = \begin{bmatrix}p&q\\r&s\end{bmatrix}$ så kan dette skrives
$$\vec v'(t) =
\begin{bmatrix}x'(t)\\y'(t)\end{bmatrix}
= \begin{bmatrix}p&q\\r&s\end{bmatrix} \cdot \begin{bmatrix}x(t)\\y(t)\end{bmatrix}
= A \cdot \begin{bmatrix}x(t)\\y(t)\end{bmatrix}
= A \cdot \vec v(t) = \vec F(t, \vec v(t)).$$

Systemet av differensialligninger kan altså skrives
$$\vec v'(t) = A \cdot \vec v(t).$$

En alternativ skrivemåte er:
$$\frac{d}{dt}\vec v(t) = A \cdot \vec v(t).$$


## Oppgave

La $A$ være en 2 x 2 matrise med egenverdier $\lambda_1$ og $\lambda_2$ og egenvektorer $\vec w_1$ og $\vec w_2$ med 
$A \cdot \vec w_1 = \lambda_1 \cdot \vec w_1$ og $A \cdot \vec w_2 = \lambda_2 \cdot \vec w_.$

La $x$ og $y$ være to tall og la $\vec v(t) = x\cdot e^{\lambda_1 t} \cdot \vec w_1 + y \cdot e^{\lambda_2 t} \cdot \vec w_2$. Sjekk at $\vec v(t)$ er en løsning til $\frac{d}{dt} \vec v(t) = A \cdot \vec v(t)$ med
$\vec v(0) = x \cdot \vec w_1 + y \cdot \vec w_2$.

## Oppgave

La 
$A=\begin{bmatrix}-1 & 2\\-3 &4\end{bmatrix}$ og la $\vec v_0 = \begin{bmatrix}1\\0\end{bmatrix}$. Finn en løsning til differensialligningssystemet $\frac{d}{dt} \vec v(t) 
= A \cdot \vec v(t)$ med $\vec v(0) = \vec v_0$ ved at følge stegene under.

1. Finn egenverdier $\lambda_1$ og $\lambda_2$ for $A$ og egenvektorer $\vec w_1$ og $\vec w_2$ med $A \cdot \vec w_1 = \lambda_1 \cdot \vec w_1$ og $A \cdot \vec w_2 = \lambda_2 \cdot \vec w_2.$
2. Finn tall $x$ og $y$ slik at $\vec v_0 = x \cdot \vec w_1 + y \cdot \vec w_2$.
3. Sjekk at $\vec v(t) = x\cdot e^{\lambda_1 t} \cdot \vec w_1 + y \cdot e^{\lambda_2 t} \cdot \vec w_2$ er en løsning til $\frac{d}{dt} \vec v(t) = A \cdot \vec v(t)$ med
$\vec v(0) = \vec v_0$.


## Komplekse egenverdier

Hvis det karakteristiske polynomet til overgangsmatrisen $A$ ikke har like mange reelle røtter som graden av polynomet kan det være fordi det finnes komplekse røtter. For eksempel har polynomet $t^2 + 1$ av grad $2$ ingen reelle røtter. Det har derimot de to komplekse røtterne $\pm i$. Det vil si at $i^2 = (-i)^2 = -1$. Python kan regne med komplekse tall. Tallet $i$ skrives `1j`.

In [2]:
import numpy as np

In [18]:
i = 1j
i * i

(-1+0j)

Hvis vi vil bruke `i * i` som et vanlig reelt tall kan vi bytte det ut med den reelle delen av tallet:

In [19]:
(i * i).real

-1.0

### Hvis det karakteristiske polynomet til overgangsmatrisen $A$ ikke har like mange **komplekse** røtter som graden av polynomet kan differensialligningssystemet $\frac{d}{dt} \vec v(t) 
= A \cdot \vec v(t)$ med $\vec v(0) = \vec v_0$ løses ved å gjøre alle utregninger med komplekse tall. 

**Advarsel** Hvis det karakteristiske polynomet er på formen $\det(A - tI) = (a_1 - t) \cdot (a_1 - t) \cdots (a_n - t)$ da må røttene $a_1, \dotsm a_n$ alle være forskjellige. Ellers feiler metoden vår. Det er mulig analytisk å løse tilfellet der røttene ikke alle er forskjellige, men det er litt knotet, så vi lar det ligge. 